In [ ]:
# prerequisite: 
# run 
#      ./angle_effect/EnergyEstimation -s ../setup/setup.json -t ../setup/target.json -d U4 -E 385.72,904.02,1843.18,2076.74,2217.45,3337.75,4089.18,5402.61 -i 17.0  > U4EE.dat
# Adopted from Erik - https://gitlab.au.dk/ausa/erik/e21010
# And modified to fit a newer iteration of the calibrations

In [2]:
import numpy as np
import uproot
import matplotlib.pyplot as plt

In [3]:
#load in generated file from above
Es = np.loadtxt('/home/haue/repositories/Masters/calibration/dssd_calibration/U4EE.dat').flatten()
print(len(Es)/4)

56.0


In [4]:
from scipy.optimize import curve_fit
def lin(x, a, b):
    return a*x + b

In [ ]:
nums =  ["4"]
sides = ["F", "B"]
Es0 = np.array([385.72,904.02, 1843.18,2076.74,2217.45,3337.75,4089.18,5402.61])
Es0_errs = np.array([0.58,1.92, 3.84, 3.84, 1.92, 4.79, 1.92, 1.92])

j = 0
for num in nums:
        for side in sides:
            for strip in range(2, 15+1, 1):
                
                print([Es[8*j+i] for i in range(len(Es0))])
                j += 1

[375.972, 897.867, 1839.36, 2073.19, 2214.04, 3335.19, 4087.0, 5400.81]
[376.119, 897.98, 1839.43, 2073.24, 2214.09, 3335.23, 4087.04, 5400.84]
[376.223, 898.065, 1839.47, 2073.28, 2214.13, 3335.26, 4087.07, 5400.86]
[376.282, 898.085, 1839.49, 2073.32, 2214.16, 3335.28, 4087.08, 5400.88]
[376.288, 898.109, 1839.51, 2073.32, 2214.17, 3335.29, 4087.09, 5400.88]
[376.249, 898.072, 1839.49, 2073.31, 2214.15, 3335.28, 4087.09, 5400.88]
[376.125, 898.006, 1839.45, 2073.28, 2214.12, 3335.24, 4087.06, 5400.86]
[375.951, 897.901, 1839.39, 2073.21, 2214.07, 3335.21, 4087.03, 5400.83]
[375.66, 897.743, 1839.3, 2073.12, 2213.97, 3335.14, 4086.97, 5400.78]
[375.272, 897.522, 1839.15, 2073.0, 2213.86, 3335.06, 4086.9, 5400.72]
[374.821, 897.222, 1838.98, 2072.83, 2213.7, 3334.95, 4086.81, 5400.65]
[374.228, 896.874, 1838.76, 2072.63, 2213.51, 3334.81, 4086.68, 5400.54]
[373.491, 896.422, 1838.48, 2072.38, 2213.26, 3334.61, 4086.52, 5400.42]
[372.469, 895.797, 1838.12, 2072.04, 2212.94, 3334.38, 408

In [ ]:
dead_strips = []

nums =  ["4"]
sides = ["F", "B"]
j = 0
res0 = res = np.array([])
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0, 0)
                    j += 1
                    continue
                # data
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                # fitted peak positions and their errors
                fx, fx_err = file[base + "F"].values()

                
                # peak positions, sorted xmin to xmax
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                fx = fx[np.argsort(fx)]
                fx_err = fx_err[np.argsort(fx)]
                
                if len(x3_sortedx)!=len(fx):
                    raise ValueError(f"Length mismatch: len(x3_sortedx)={len(x3_sortedx)} is not equal to len(fx)={len(fx)}, somehow the fitting procedure messed up..")



                # proton peaks pV, pVI, ..., pVIII
                while len(x3_sortedx)>len(Es0):
                    min_y_index = np.argmin(y3_sortedx)
                    x3_sortedx = np.delete(x3_sortedx,min_y_index)
                    y3_sortedx = np.delete(y3_sortedx,min_y_index)
                    fx = np.delete(fx,min_y_index)
                    fx_err = np.delete(fx_err,min_y_index)
                    
                
                ### the gaussian fitting routine is very simple and therefore one should expect errors
                ### This would explain why x3_sortedx is the better parameter to use for now
                #chs = fx
                chs = x3_sortedx
                popt0, pcov0 = curve_fit(lin, chs, Es0, sigma=Es0_errs, p0=[2, 0])
                Ecs = np.array([Es[8*j+i] for i in range(len(x3_sortedx))])
                popt, pcov = curve_fit(lin, chs, Ecs, sigma=Es0_errs, p0=[2, 0])
                
                res0 = np.append(res0, Es0 - lin(chs, *popt0)) ## not energy corrected peak energies
                res  = np.append(res, np.sort(Ecs) - lin(chs, *popt)) ## energy corrected peak energies ! the most important one!
                
                ######################################################################################################################
                ## plotting routine for quality control
                """
                fig, axes = plt.subplots(2, 1,gridspec_kw={'height_ratios': [4, 1], 'hspace': 0}, sharex=True)
                ax1, ax2 = axes
                #plt.sca(axes[0]) ## sets current axes
                ax1.errorbar(chs, Ecs, yerr=Es0_errs, capsize=2, marker='o',markersize=3.5, ls='', color='r', label="Data points")
                ax1.plot([50, 2000], lin(np.array([50, 2000]), *popt), label=f"fit vals: {np.round(popt[1],6)} {np.round(popt[0],6)}", color='k', ls='--')
                ax1.legend()
                ax1.set_ylabel("Energy(keV)")
                ax1.set_title(f"Linear fit to {base}")
                #plt.sca(axes[1])
                ax2.scatter(chs, Ecs - lin(chs, *popt), color='k', s=8)
                ax2.hlines(0,50,2000, ls='--', color='k', lw=0.7)
                ax2.set_yticks(np.arange(-10,10, 2.5), minor=True)
                ax2.set_ylim(-np.max(np.abs(Ecs - lin(chs, *popt)))-2, np.max(np.abs(Ecs - lin(chs, *popt)))+2)
                
                ax2.set_ylabel("Residuals (keV)")
                ax2.set_xlabel("Channel")
                
                plt.show()
                
                
                """
                
                ###Print statement for calibrations
                #print(popt[1], popt[0])      
                
                ###Print statement to define calibration errors
                #print(np.sqrt(pcov[1,1]), np.sqrt(pcov[0,0]), pcov[0,1])
                
                ### Print statement used for plotting in compare.ipynb
                if base=="U4F7":
                    print(x3_sortedx)
                    print(popt[1], popt[0])
                    print(np.sort(Ecs) - lin(chs, *popt))
                    
                    
                j += 1


# Front
0 0 0
[ 113.99539713  256.99103479  514.00189975  577.00414222  616.00154576
  921.99575502 1127.99984529 1487.99644195]
-40.70991056872542 3.658247711535433
[-0.06449011 -1.35495435 -0.14636292  3.19582774  1.37366547  3.10104972
  1.29705793 -1.86966797]
0 0 0
# Back
0 0 0
0 0 0
